# Project Title
### Data Engineering Capstone Project

#### Project Summary
This project is based on Anime dataset that were obtained through 3 sets one for user level, another for Anime level and third describing the status, dataset is going to be loaded into OLAP for seek of analysis and  building  Anime recommendation system later, the data raw format is in csv and it is going to be loaded in querable tables  

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import cleansing_functions as clean
import os
import configparser
import datetime as dt
from pyspark.sql.functions import isnan, when, count, col, udf, dayofmonth, dayofweek, month, year, weekofyear, avg, monotonically_increasing_id
from pyspark.sql.types import *
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType, StringType, IntegerType, FloatType,DateType
from pyspark.sql import functions as F
import create_tables as ct
import Quality_Checks as qc

### Step 1: Scope the Project and Gather Data

#### Scope 
 I am going to explore, clean data by removing duplicates and unneeded columns, orient it in relational database to help data scientist train model for building Anime recommendation system that can be used for advertising and helping users find their next Anime to watch

#### Describe and Gather Data 
I got the data from Kaggle, it is describing some measures about anime viewers like how many episodes he/she watched and anime measures like ratings, count of views and when it was first shown, etc.. 
aslo there was table for watching status mapping and also I added date table that I generated with DAX query and attached the screenshot on the left, it could be done using spark also but was easier for me do it through DAX

In [2]:
anime_df = pd.read_csv('anime.csv')

In [3]:
anime_df.head()

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0


In [4]:
anime_df.dtypes

MAL_ID            int64
Name             object
Score            object
Genres           object
English name     object
Japanese name    object
Type             object
Episodes         object
Aired            object
Premiered        object
Producers        object
Licensors        object
Studios          object
Source           object
Duration         object
Rating           object
Ranked           object
Popularity        int64
Members           int64
Favorites         int64
Watching          int64
Completed         int64
On-Hold           int64
Dropped           int64
Plan to Watch     int64
Score-10         object
Score-9          object
Score-8          object
Score-7          object
Score-6          object
Score-5          object
Score-4          object
Score-3          object
Score-2          object
Score-1          object
dtype: object

In [5]:
Anime_Dict = pd.read_csv('data dictionary for Anime.csv')


#### Data Dictionary for Anime dataset

In [6]:
Anime_Dict.head(100)

,Column,Description
0,MAL_ID,MyAnimelist ID of the anime. (e.g. 1)
1,Name,full name of the anime. (e.g. Cowboy Bebop)
2,Score,average score of the anime given from all use...
3,Genres,comma separated list of genres for this anime...
4,English name,full name in english of the anime. (e.g. Cowb...
5,Japanese name,full name in japanses of the anime. (e.g. カウボ...
6,Type,"TV, movie, OVA, etc. (e.g. TV)"
7,Episodes',number of chapters. (e.g. 26)
8,Aired,"broadcast date. (e.g. Apr 3, 1998 to Apr 24, ..."
9,Premiered,season premiere. (e.g. Spring 1998)


In [7]:
anime_user_rating_df = pd.read_csv('anime user rating.csv')

In [8]:
anime_user_rating_df.head()

,user_id,anime_id,rating,watching_status,watched_episodes
0,0,67,9,1,1
1,0,6702,7,1,4
2,0,242,10,1,4
3,0,4898,0,1,1
4,0,21,10,1,0


In [9]:
User_rating_Dict = pd.read_csv('Anime user rating dictionary.csv')

#### Data Dictionary for Anime user rating

In [10]:
User_rating_Dict.head() 

,Column,Description
0,user_id,non identifiable randomly generated user id.
1,anime_id,MyAnemlist ID of the anime. (e.g. 1).
2,score,score between 1 to 10 given by the user. 0 if...
3,watching_status,state ID from this anime in the anime list of...
4,watched_episodes,numbers of episodes watched by the user. (e.g...


In [11]:
watching_status_df = pd.read_csv('watching_status.csv')

In [12]:
watching_status_df.head()

,status,description
0,1,Currently Watching
1,2,Completed
2,3,On Hold
3,4,Dropped
4,6,Plan to Watch


In [13]:
date_df =  pd.read_csv('date.csv')

In [14]:
date_df.head()

,Date,Calendar Year,Month Name,Month Number,Quarter,ID
0,01/07/1900,1900,July,7,Q3,182
1,02/07/1900,1900,July,7,Q3,183
2,03/07/1900,1900,July,7,Q3,184
3,04/07/1900,1900,July,7,Q3,185
4,05/07/1900,1900,July,7,Q3,186


### Step 2: Explore and Assess the Data
#### Explore the Data 
After exploring anime dataframe, I noticed that:
1. we have multiple series column that we may care for the first value more for ease of model training
2. we have series column that need to be split for considering separate values
3. date column is not in proper format that we can filter using date column

#### Cleaning Steps
In the following steps I will be handing each issue of the above mentioned using cleaning functions,
that I defined in the script cleaning_functions and imported at the begining of the notebook

In [15]:
cleaned_anime_df = clean.clean_dataset(df=anime_df,
                                         series_columns = ['Producers','Studios','Licensors'],
                                         columns_to_delete = ['Producers','Studios','Licensors','Premiered'],
                                         columns_to_split=['Genres'],
                                        splits = 3,
                                        splitter = ',')

null records were dropped from dataset
Duplicate records were dropped from dataset
list column Producers was converted to string by considering first value of list
list column Studios was converted to string by considering first value of list
list column Licensors was converted to string by considering first value of list
columns ['Producers', 'Studios', 'Licensors', 'Premiered'] were dropped
column Genres was split into 3 columns


In [16]:
cleaned_anime_df = clean.adjust_date_column(df=cleaned_anime_df,date_column = 'Aired',new_date ='First Show' )

string column Aired was replaced with date column First Show


In [17]:
cleaned_anime_df.head()

,MAL_ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Source,Duration,...,Score-3,Score-2,Score-1,Main Producers,Main Studios,Main Licensors,Genres1,Genres2,Genres3,First Show
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,Original,24 min. per ep.,...,1357.0,741.0,1580.0,Bandai Visual,Sunrise,Funimation,Action,Adventure,Comedy,1998-04-03
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,Original,1 hr. 55 min.,...,221.0,109.0,379.0,Sunrise,Bones,Sony Pictures Entertainment,Action,Drama,Mystery,2001-09-01
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,Manga,24 min. per ep.,...,664.0,316.0,533.0,Victor Entertainment,Madhouse,Funimation,Action,Sci-Fi,Adventure,1998-04-01
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,Original,25 min. per ep.,...,353.0,164.0,131.0,TV Tokyo,Sunrise,Funimation,Action,Mystery,Police,2002-07-02
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,Manga,23 min. per ep.,...,83.0,50.0,27.0,TV Tokyo,Toei Animation,Unknown,Adventure,Fantasy,Shounen,2004-09-30


In [18]:
cleaned_anime_user_rating_df=clean.clean_dataset(df=anime_user_rating_df)

null records were dropped from dataset
Duplicate records were dropped from dataset


In [19]:
cleaned_anime_user_rating_df.head()

,user_id,anime_id,rating,watching_status,watched_episodes
0,0,67,9,1,1
1,0,6702,7,1,4
2,0,242,10,1,4
3,0,4898,0,1,1
4,0,21,10,1,0


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
<img src="b.jpg" alt="Alternative text" />


#### 3.2 Mapping Out Data Pipelines
##### To get data into the above schema,
data should be loaded into staging tables

Create Dimension and Facts tables

insert into dimensions and facts

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

#### Load into staging

In [20]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [21]:
anime_schema = StructType([StructField("MAL_ID", IntegerType(), True)\
                          ,StructField("Name", StringType(), True)\
                          ,StructField("Score", StringType(), True)\
                          ,StructField("Genres", StringType(), True)\
                          ,StructField("English_Name", StringType(), True)\
                          ,StructField("Japanese_Name", StringType(), True)\
                          ,StructField("Type", StringType(), True)\
                          ,StructField("Epoisodes", StringType(), True)\
                          ,StructField("Source", StringType(), True)\
                          ,StructField("Duration", StringType(), True)\
                          ,StructField("Rating", StringType(), True)\
                          ,StructField("Ranked", StringType(), True)\
                          ,StructField("Popularity", StringType(), True)\
                          ,StructField("Members", StringType(), True)\
                          ,StructField("Favorites",StringType(), True)\
                          ,StructField("Watching",StringType(), True)\
                          ,StructField("Completed", StringType(), True)\
                          ,StructField("On-Hold", StringType(), True)\
                          ,StructField("Dropped", StringType(), True)\
                          ,StructField("Plan_to_Watch", StringType(), True)\
                          ,StructField("Score-10", StringType(), True)\
                          ,StructField("Score-1", StringType(), True)\
                          ,StructField("Score-2", StringType(), True)\
                          ,StructField("Score-3", StringType(), True)\
                          ,StructField("Score-4", StringType(), True)\
                          ,StructField("Score-5", StringType(), True)\
                          ,StructField("Score-6", StringType(), True)\
                          ,StructField("Score-7", StringType(), True)\
                          ,StructField("Score-8", StringType(), True)\
                          ,StructField("Score-9", StringType(), True)\
                          ,StructField("Main_Producers", StringType(), True)\
                          ,StructField("Main_Studios", StringType(), True)\
                          ,StructField("Main_Licensors", StringType(), True)\
                          ,StructField("Genres1", StringType(), True)\
                          ,StructField("Genres2", StringType(), True)\
                          ,StructField("Genres3", StringType(), True)\
                          ,StructField("First_Show", DateType(), True)])
anime_spark = spark.createDataFrame(cleaned_anime_df, schema=anime_schema)

In [22]:
anime_spark.toPandas().head()

,MAL_ID,Name,Score,Genres,English_Name,Japanese_Name,Type,Epoisodes,Source,Duration,...,Score-7,Score-8,Score-9,Main_Producers,Main_Studios,Main_Licensors,Genres1,Genres2,Genres3,First_Show
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,Original,24 min. per ep.,...,1357.0,741.0,1580.0,Bandai Visual,Sunrise,Funimation,Action,Adventure,Comedy,1998-04-03
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,Original,1 hr. 55 min.,...,221.0,109.0,379.0,Sunrise,Bones,Sony Pictures Entertainment,Action,Drama,Mystery,2001-09-01
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,Manga,24 min. per ep.,...,664.0,316.0,533.0,Victor Entertainment,Madhouse,Funimation,Action,Sci-Fi,Adventure,1998-04-01
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,Original,25 min. per ep.,...,353.0,164.0,131.0,TV Tokyo,Sunrise,Funimation,Action,Mystery,Police,2002-07-02
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,Manga,23 min. per ep.,...,83.0,50.0,27.0,TV Tokyo,Toei Animation,Unknown,Adventure,Fantasy,Shounen,2004-09-30


In [23]:
anime_viewer_schema = StructType([StructField("User_ID", IntegerType(), True)\
                          ,StructField("Anime_ID", IntegerType(), True)\
                          ,StructField("Rating", IntegerType(), True)\
                          ,StructField("Watching_Status_ID", IntegerType(), True)\
                          ,StructField("Watched_Episodes",IntegerType(), True)])
anime_viewer_spark = spark.createDataFrame(cleaned_anime_user_rating_df, schema=anime_viewer_schema )

In [24]:
anime_viewer_spark.toPandas().head()

,User_ID,Anime_ID,Rating,Watching_Status_ID,Watched_Episodes
0,0,67,9,1,1
1,0,6702,7,1,4
2,0,242,10,1,4
3,0,4898,0,1,1
4,0,21,10,1,0


In [25]:
date_schema = StructType([StructField("Date", StringType(), True)\
                          ,StructField("Year", IntegerType(), True)\
                          ,StructField("Month_Name", StringType(), True)\
                          ,StructField("Month", IntegerType(), True)\
                          ,StructField("Quarter", StringType(), True)\
                         ,StructField("ID", IntegerType(), True)])
date_spark = spark.createDataFrame(date_df,schema = date_schema)

In [26]:
date_spark.toPandas().head()

,Date,Year,Month_Name,Month,Quarter,ID
0,01/07/1900,1900,July,7,Q3,182
1,02/07/1900,1900,July,7,Q3,183
2,03/07/1900,1900,July,7,Q3,184
3,04/07/1900,1900,July,7,Q3,185
4,05/07/1900,1900,July,7,Q3,186


In [27]:
status_schema = StructType([StructField("Status_ID", IntegerType(), True)\
                          ,StructField("Status_Description", StringType(), True)])

status_spark = spark.createDataFrame(watching_status_df,schema =status_schema)

In [28]:
status_spark.toPandas().head()

,Status_ID,Status_Description
0,1,Currently Watching
1,2,Completed
2,3,On Hold
3,4,Dropped
4,6,Plan to Watch


In [29]:
status_spark.toPandas().columns

Index(['Status_ID', 'Status_Description'], dtype='object')

#### Load into Dimensions and Facts

In [30]:
output_path = "tables/"

In [31]:
ct.create_dim_anime(anime_spark, output_path)

Table DimAnime was witten successfully to tables/DimAnime


In [32]:
DimAnime = spark.read.parquet("tables/DimAnime")
DimAnime.toPandas().head()

,MAL_ID,Name,English_Name,Japanese_Name,Type,Main_Studios,Main_Producers,Main_Licensors,Source,Rating,Genres1,Genres2,Genres3
0,57,Beck,Beck:Mongolian Chop Squad,ベック,TV,Madhouse,Marvelous,Funimation,Manga,R - 17+ (violence & profanity),Comedy,Drama,Music
1,570,Jin-Rou,Jin-Roh:The Wolf Brigade,人狼 JIN-ROH,Movie,Production I.G,Unknown,Bandai Entertainment,Manga,R - 17+ (violence & profanity),Military,Police,Psychological
2,868,Slayers Great,Slayers Movie 3,スレーヤーズぐれえと,Movie,J.C.Staff,TV Tokyo,ADV Films,Light novel,PG-13 - Teens 13 or older,Adventure,Comedy,Fantasy
3,920,Nurse Witch Komugi-chan Magikarte Special,Unknown,閑話休題すぺしゃる ～再び祭りの予感でワッショイ!～,Special,Kyoto Animation,Rondo Robe,ADV Films,Original,PG-13 - Teens 13 or older,Comedy,Magic,Parody
4,1026,Yakumotatsu,Eight Clouds Rising,八雲立つ,OVA,Studio Pierrot,Bandai Visual,Media Blasters,Manga,PG-13 - Teens 13 or older,Adventure,Supernatural,Historical


In [33]:
ct.create_dim_anime_viewer(anime_viewer_spark, output_path)

Table DimAnimeViewer was witten successfully to tables/DimAnimeViewer


In [34]:
DimAnimeViewer = spark.read.parquet("tables/DimAnimeViewer")
DimAnimeViewer.toPandas().head()

,User_ID,Watched_Episodes
0,98,222
1,195,77
2,256,1
3,678,7
4,714,5


In [35]:
ct.create_dim_watching_status(status_spark, output_path)

Table DimWathcingStatus was witten successfully to tables/DimWathcingStatus


In [36]:
DimWathcingStatus = spark.read.parquet("tables/DimWathcingStatus")
DimWathcingStatus.toPandas().head()

,Status_ID,Status_Description
0,1,Currently Watching
1,6,Plan to Watch
2,2,Completed
3,3,On Hold
4,4,Dropped


In [37]:
ct.create_dim_date(date_spark, output_path)

Table DimDate was witten successfully to tables/DimDate


In [38]:
DimDate = spark.read.parquet("tables/DimDate")
DimDate.toPandas().head()

,Date,Year,Month_Name,Month,Quarter,ID,Formatted_Date
0,15/05/1900,1900,May,5,Q2,135,1900-05-15
1,05/09/1900,1900,September,9,Q3,248,1900-09-05
2,14/05/1901,1901,May,5,Q2,499,1901-05-14
3,06/06/1901,1901,June,6,Q2,522,1901-06-06
4,21/06/1902,1902,June,6,Q2,902,1902-06-21


In [39]:
ct.create_fact_user_rating(anime_viewer_spark, output_path)

Table FactUserRating was witten successfully to tables/FactUserRating


In [40]:
FactUserRating = spark.read.parquet("tables/FactUserRating")
FactUserRating.toPandas().head()

,User_ID,Anime_ID,Rating,Watching_Status_ID
0,2,6893,0,2
1,2,9588,0,6
2,4,393,5,2
3,5,617,5,2
4,6,4715,9,2


In [41]:
ct.create_fact_anime_measures(spark_df = anime_spark,dim_date = DimDate,output_path = output_path)

Table FactAnimeMeasures was witten successfully to tables/FactAnimeMeasures


In [42]:
FactAnimeMeasures = spark.read.parquet("tables/FactAnimeMeasures")
FactAnimeMeasures.toPandas().head()

,MAL_ID,Score,First_Show_Date_ID,Duration,Ranked,Popularity,Members,Favorites,Watching,Completed,...,Score-10,Score-1,Score-2,Score-3,Score-4,Score-5,Score-6,Score-7,Score-8,Score-9
0,77,7.99,38626.0,25 min. per ep.,551.0,2075,56981,1014,1367,40165,...,4189,6809,9368,6267,2220,864,250,83,53,71
1,102,7.48,38079.0,25 min. per ep.,1634.0,1529,89533,571,3772,59770,...,4497,7282,12321,12618,6389,3227,1069,426,187,180
2,361,5.93,37215.0,30 min. per ep.,8448.0,6120,5715,4,162,2622,...,74,87,186,443,461,352,211,94,58,36
3,589,8.04,31508.0,23 min. per ep.,486.0,3568,21057,751,721,11688,...,2130,1810,2201,1830,746,353,134,61,36,36
4,814,7.49,38380.0,30 min.,1618.0,4355,13428,44,135,11041,...,923,989,1642,1940,919,430,126,44,30,23


#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

##### Check Uniqueness of key column

In [43]:
tables = [
    [DimAnime,"DimAnime","MAL_ID"],
    [DimAnimeViewer,"DimAnimeViewer","User_ID"],
    [DimWathcingStatus,"DimWathcingStatus","Status_ID"],
    [DimDate,"DimDate","ID"],
    [FactUserRating,"FactUserRating",["User_ID","Anime_ID"]],
    [FactAnimeMeasures,"FactAnimeMeasures", "MAL_ID"]
]
for spark_df,table_name,PK in tables:
    qc.check_uniqueness(spark_df,table_name,PK )

uniqeness check passed for table DimAnime unique column MAL_ID
uniqeness check passed for table DimAnimeViewer unique column User_ID
uniqeness check passed for table DimWathcingStatus unique column Status_ID
uniqeness check passed for table DimDate unique column ID
uniqeness check passed for table FactUserRating unique column ['User_ID', 'Anime_ID']
uniqeness check passed for table FactAnimeMeasures unique column MAL_ID


##### Check table records count

In [44]:
tables = {
    DimAnime:"DimAnime",
    DimAnimeViewer:"DimAnimeViewer",
    DimWathcingStatus :"DimWathcingStatus",
    DimDate :"DimDate",
    FactUserRating : "FactUserRating",
    FactAnimeMeasures : "FactAnimeMeasures"
}
for table,table_name in tables.items():
    qc.check_table_record_count(table,table_name)

Data quality check passed for DimAnime with 17562 records
Data quality check passed for DimAnimeViewer with 3292 records
Data quality check passed for DimWathcingStatus with 5 records
Data quality check passed for DimDate with 44926 records
Data quality check passed for FactUserRating with 1048575 records
Data quality check passed for FactAnimeMeasures with 17562 records


##### Check Completness between source and destination

In [45]:
tables = [
    [DimAnime,anime_spark,"DimAnime","MAL_ID"],
    [DimAnimeViewer,anime_viewer_spark,"DimAnimeViewer","User_ID"],
    [DimWathcingStatus,status_spark,"DimWathcingStatus","Status_ID"],
    [DimDate,date_spark,"DimDate","ID"],
    [FactUserRating,anime_viewer_spark,"FactUserRating",["User_ID","Anime_ID"]],
    [FactAnimeMeasures,anime_spark,"FactAnimeMeasures", "MAL_ID"]
]
for spark_table,source_df,PK,table_name in tables:
    qc.check_source_Destination_count(spark_table,source_df,PK,table_name)

Completness check passed for final table DimAnime compared to sataging
Completness check passed for final table DimAnimeViewer compared to sataging
Completness check passed for final table DimWathcingStatus compared to sataging
Completness check passed for final table DimDate compared to sataging
Completness check passed for final table FactUserRating compared to sataging
Completness check passed for final table FactAnimeMeasures compared to sataging


#### 4.3 Data dictionary 
you can refer to step 1 for each importe dataset it is followed by its dictionary

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

#### Clearly state the rationale for the choice of tools and technologies for the project.
we used for data processing Apache spark, being one of best choices for big data ingestion and analysis,
as well as its integration with python libraries.
I used for raw data manipulation,cleansing andinvestigation pandas library due to its variaous functions and ease of data manipulation

#### Propose how often the data should be updated and why.
I think this data may be updated monthly,daily or even hourly according to data scientist requirement for model data feeding and retraining to get best recommendation for website users

#### If data is to be increased by 100x,
we should manage our resources, storage and memory either horizontally be adding nodes or verically be elevating our cluster power, parallel processing will be effective also using 
cloud unlimiited scalable storage and processing capabilities
will be a good choice like amazon redshift for OLAP and EMR for processing

#### If data should be populated on daily basis,
This can be handled simply by defining apache Airflow dag with defined cron job, setting tasks dependancy
and it can simply handle processing data d=on daily basis

#### If database needed to be accessed by 100+ people.
 at first we should verify that OLAP is separated from OLTP 
 using indices should be considered for fast retrieval of data
 ensure denormalization of data to decrease need for joins
 we can benefit from auto scalability and resizing feature for Amazon Redshift